In [1]:
import tensorflow as tf
import numpy as np
import pickle as cp
from sklearn.preprocessing import OneHotEncoder

In [2]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print("shapes: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)
    
    print('Dataset loaded successfully')

    return X_train, y_train, X_test, y_test

In [3]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [4]:
def segment_data(X_train, y_train, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(test_segments), np.array(test_labels)

# Network params and data elaboration


In [5]:
num_sensors = 113 # number of sensor channels
num_classes = 18 # number of classes 
window_size = 24 # window size
step_size = 12 # half of the sliding window

In [6]:
X_train, y_train, X_test, y_test = load_dataset('oppChallenge_gestures.data')

train_segments, train_labels, test_segments, test_labels = segment_data(X_train, y_train, X_test, y_test,
                                                                        window_size, step_size)

shapes: train (557963, 113), test (118750, 113)
Dataset loaded successfully


In [7]:
print(train_segments.shape)

(46495, 24, 113)


In [8]:
# one-hot encoding of labels

encoder = OneHotEncoder()
train_labels = encoder.fit_transform(train_labels.reshape(-1,1)).toarray()
test_labels = encoder.transform(test_labels.reshape(-1,1)).toarray()

In [9]:
print(train_labels.shape)
train_labels

(46495, 18)


array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [10]:
# understanding batches
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# CNN + LSTM network

In [11]:
def conv2d_layer(prev_layer, num_filters, kernel, strides):
    return tf.layers.conv2d(prev_layer, num_filters, kernel, strides=strides, activation=tf.nn.relu)

In [12]:
def lstm_layers(prev_layer, lstm_size):
    rnn_layers = [tf.nn.rnn_cell.BasicLSTMCell(size, activation=tf.nn.tanh) for size in [lstm_size, lstm_size]]
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    
    return tf.nn.dynamic_rnn(cell=multi_rnn_cell, inputs=prev_layer, dtype=tf.float32)

In [13]:
def dropout_layer(prev_layer, hold_prob):
    return tf.nn.dropout(prev_layer, keep_prob=hold_prob)

In [14]:
conv_output = 8
num_filters = 64
lstm_size = 128
kernel_size = (5,1)
strides = 1

In [15]:
X = tf.placeholder(tf.float32, shape=[None, window_size, num_sensors, 1])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes])
hold = tf.placeholder(tf.float32)

In [16]:
conv_1 = conv2d_layer(X, num_filters, kernel_size, strides)
conv_2 = conv2d_layer(conv_1, num_filters, kernel_size, strides)
conv_3 = conv2d_layer(conv_2, num_filters, kernel_size, strides)
conv_4 = conv2d_layer(conv_3, num_filters, kernel_size, strides)

In [17]:
drop_1 = dropout_layer(conv_4, hold)

In [18]:
reshaped = tf.reshape(drop_1, [-1, conv_output, num_filters*num_sensors])

In [19]:
lstm_output, states = lstm_layers(reshaped, lstm_size)

In [20]:
last_output = lstm_output[:,-1,:]

In [21]:
drop_2 = dropout_layer(last_output, hold)

In [22]:
y_pred = tf.layers.dense(drop_2, num_classes, kernel_initializer=tf.initializers.truncated_normal(stddev=0.1),
                          bias_initializer=tf.initializers.constant(0.1))

In [23]:
# cost function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [24]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [25]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [26]:
epochs = 50
batchSize = 100
hold_prob = 0.5

with tf.Session() as sess:
    
    sess.run(init)
    
    for ep in range(epochs):
        
        for batch in iterate_minibatches(train_segments, train_labels, batchSize, True):
            
            batch_x, batch_y = batch
            batch_x = np.reshape(batch_x, (-1,window_size,num_sensors,1))
            sess.run(train,feed_dict={X:batch_x, y_true:batch_y, hold:hold_prob})
        
        i = 0
        for batch in iterate_minibatches(test_segments, test_labels, batchSize):
            
            batch_x, batch_y = batch
            batch_x = np.reshape(batch_x, (-1,window_size,num_sensors,1))
            
            pred = y_pred.eval(feed_dict={X:batch_x, y_true:batch_y, hold:1.0})
  
            matches = np.equal(np.argmax(pred,1),np.argmax(batch_y,1))
            matches = matches.astype(np.float32)
         
            if i == 0:
                tot_matches = matches
            else:    
                tot_matches = np.concatenate([tot_matches, matches], axis=0)
            i=i+1

        print('Currently on epoch {}'.format(ep+1))
        print('Test accuracy is: {}'.format(np.mean(tot_matches)))
        print('\n')
        del tot_matches
            
saver.save(sess, "./CNN_LSTM_model")
        
                

Currently on epoch 1
Test accuracy is: 0.8597959280014038


Currently on epoch 2
Test accuracy is: 0.8645918369293213


Currently on epoch 3
Test accuracy is: 0.881224513053894


Currently on epoch 4
Test accuracy is: 0.8650000095367432


Currently on epoch 5
Test accuracy is: 0.8886734843254089


Currently on epoch 6
Test accuracy is: 0.8787755370140076


Currently on epoch 7
Test accuracy is: 0.8839796185493469


Currently on epoch 8
Test accuracy is: 0.8953061103820801


Currently on epoch 9
Test accuracy is: 0.8827551007270813


Currently on epoch 10
Test accuracy is: 0.8922448754310608


Currently on epoch 11
Test accuracy is: 0.8901020288467407


Currently on epoch 12
Test accuracy is: 0.867959201335907


Currently on epoch 13
Test accuracy is: 0.8928571343421936


Currently on epoch 14
Test accuracy is: 0.884591817855835


Currently on epoch 15
Test accuracy is: 0.8974489569664001


Currently on epoch 16
Test accuracy is: 0.8657143115997314


Currently on epoch 17
Test accuracy 

RuntimeError: Attempted to use a closed Session.